# 1. Data Extraction from Wikipedia, clean and display the top 10 rows along with shape head

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
# download url data from internet
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
Canada_data = BeautifulSoup(source, 'lxml')

In [4]:
# creat a new Dataframe
column_names = ['Postalcode','Borough','Neighborhood']
toronto = pd.DataFrame(columns = column_names)

In [42]:
content = Canada_data.find('div', class_='mw-parser-output')
table = content.table.tbody
postcode = 0
borough = 0
neighborhood = 0

for tr in table.find_all('tr'):
    i = 0
    for td in tr.find_all('td'):
        if i == 0:
            postcode = td.text
            i = i + 1
        elif i == 1:
            borough = td.text
            i = i + 1
        elif i == 2: 
            neighborhood = td.text.strip('\n').replace(']','')
    toronto = toronto.append({'Postalcode': postcode,'Borough': borough,'Neighborhood': neighborhood},ignore_index=True)

# clean dataframe 
toronto = toronto[toronto.Borough!='Not assigned\n']
toronto = toronto[toronto.Borough!= 0]
toronto.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,toronto.shape[0]):
    if toronto.iloc[i][2] == 'Not assigned':
        toronto.iloc[i][2] = toronto.iloc[i][1]
        i = i+1
                                 
df = toronto.groupby(['Postalcode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
df.head()

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae"


# Data Cleaning

In [46]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern / Rouge, Malvern / Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E\n,Scarborough\n,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G\n,Scarborough\n,"Woburn, Woburn"
4,M1H\n,Scarborough\n,"Cedarbrae, Cedarbrae"


In [121]:
df['Postalcode'] = df['Postalcode'].str.replace('\n', '')
df['Borough'] = df['Borough'].str.replace('\n', '')
df.head()

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [122]:
def neighborhood_list(grouped):    
    return ', '.join(sorted(grouped['Neighborhood'].tolist()))
                    
grp = df.groupby(['Postalcode', 'Borough'])
df2 = grp.apply(neighborhood_list).reset_index(name='Neighborhood')

In [123]:
print(df2.shape)
df2.head()

(103, 3)


,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge"
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G,Scarborough,"Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae"


In [129]:

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_540b4e2799614e2fa8b941826121d98e = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='5-ur7DuZ9n4gwm9Spk6WDFHPBxuDII4vu2PZ8heEAc2q',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body = client_540b4e2799614e2fa8b941826121d98e.get_object(Bucket='capstoneproject-donotdelete-pr-xml2fcdlu1ll6e',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df_geocsv_data = pd.read_csv(body)

df_geocsv_data.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [132]:
print(df_geocsv_data.shape)

(103, 3)


In [133]:
df3= pd.merge(df_geocsv_data, df, on = 'Postalcode')
df3.head()

,Postalcode,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern / Rouge, Malvern / Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug..."
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood / Morningside / West Hill, Guildwood..."
3,M1G,43.770992,-79.216917,Scarborough,"Woburn, Woburn"
4,M1H,43.773136,-79.239476,Scarborough,"Cedarbrae, Cedarbrae"


In [144]:
df = df3[['Postalcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']]
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern / Rouge, Malvern / Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek, Roug...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood / Morningside / West Hill, Guildwood...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn, Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae, Cedarbrae",43.773136,-79.239476


In [145]:
df.to_csv('Toronto.TASK_II_df.csv',index=False)

In [147]:
df.shape

(103, 5)

In [171]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [177]:
mkdir visualization
cd visualization

SyntaxError: invalid syntax (<ipython-input-177-0ce135a922b6>, line 1)

In [179]:
https://github.com/python-visualization/folium

SyntaxError: invalid syntax (<ipython-input-179-a7f636ae4101>, line 1)

In [180]:
cd folium
python setup.py install

SyntaxError: invalid syntax (<ipython-input-180-e421ad5f0289>, line 1)

In [152]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [174]:
"conda install folium"
print('installed')

installed


In [175]:
import folium

ModuleNotFoundError: No module named 'folium'

In [164]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

NameError: name 'folium' is not defined